This is technically using the telephone environment

## Huggingface dataset version of childes

In [1]:
import os
from os.path import join, exists

from transformers import BertForMaskedLM, BertTokenizer
import datasets
from datasets import load_dataset

import importlib

ImportError: No module named 'transformers'

In [ ]:


dataset_path = 'w-nicole/childes_data'
childes_data = load_dataset(dataset_path)

#2/20: https://huggingface.co/transformers/quickstart.html

model = BertForMaskedLM.from_pretrained('bert-large-uncased-whole-word-masking')
model.eval()
   
tokenizer = BertTokenizer.from_pretrained("bert-large-uncased-whole-word-masking")
   

# 6/3: https://github.com/huggingface/notebooks/blob/master/examples/language_modeling.ipynb

def tokenize_function(examples): 
    return tokenizer(examples["text"])

tokenized_datasets = {}
for phase in ['train', 'validation']:
    tokenized_datasets[phase] = childes_data[phase].map(tokenize_function, batched=True)
    
    # If you get NonMatchingSplitsSizesError
    # 6/3: https://github.com/huggingface/datasets/issues/215

Reusing dataset childes (/home/nwong/.cache/huggingface/datasets/childes/childes_data/1.0.0/afc4079a0afe4a4f7cc7d6a4698dce6946c1866efede8ea4307b8ee994b80ffa)
Some weights of the model checkpoint at bert-large-uncased-whole-word-masking were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
for i in range(5):
    print(childes_data['train'][i])
    
print(); print()

for i in range(5):
    print(childes_data['validation'][i])

In [ ]:

# Development of the training code 
    
# Code is taken on 6/3 from here:
# https://colab.research.google.com/github/huggingface/blog/blob/master/notebooks/01_how_to_train.ipynb#scrollTo=zTgWPa9Dipk2

from transformers import DataCollatorForWholeWordMask
from transformers import Trainer, TrainingArguments

data_collator = DataCollatorForWholeWordMask(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

# Try to memorize a single example as a sanity check

training_args = TrainingArguments(
    output_dir="./test_bert_fit",
    overwrite_output_dir=True,
    num_train_epochs=100,
    save_steps=10_000,
    save_total_limit=2,
    prediction_loss_only=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    tokenizer = tokenizer,
    train_dataset=tokenized_datasets['train'],
    eval_dataset = tokenized_datasets['validation'],
    
    # This is a sanity check
)



In [ ]:
# Initial performance -- trying to check if the model fitted at all

trainer.evaluate()

In [ ]:
#Still from 6/3: https://colab.research.google.com/github/huggingface/blog/blob/master/notebooks/01_how_to_train.ipynb#scrollTo=VmaHZXzmkNtJ


results = trainer.train()

In [ ]:
trainer.evaluate()

# It's not very obvious if it's fitting or not?

# BERT trains! (6/4)
# Am changing back the train/val

## Exploration code